In [6]:
import Pkg
using EzXML
using XMLDict
using Dates

In [9]:
#include("src/test_name.jl")
#using .Test_name


In [10]:
"""
getAnnotationPaths(safePath::string)

Getting the paths for the annotation files for a SLC image using its .SAFE folder path.

### Parameters
    * safePath::String: path of .SAFE folder for one image.
    
### Returns
    * annotationPaths::Vector: Vector of paths for annotation files in .SAFE folder
"""
function getAnnotationPaths(safePath::String)::Vector{String}
    annotationFolder = joinpath(safePath, "annotation")
    return [joinpath(annotationFolder, annotationFile) for annotationFile in searchDir(annotationFolder, ".xml")]
end


"""
getDataPaths(safePath::string)

Getting the paths for the annotation files for a SLC image using its .SAFE folder path.

### Parameters
    * safePath::String: path of .SAFE folder for one image.
    
### Returns
    * annotationPaths::Vector: Vector of paths for annotation files in .SAFE folder
"""
function getDataPaths(safePath::String)::Vector{String}
    annotationFolder = joinpath(safePath, "measurement")
    return [joinpath(annotationFolder, annotationFile) for annotationFile in searchDir(annotationFolder, ".tiff")]
end



""""

search dir

Searching a directory for files with extention.

"""
searchDir(path, key) = filter(x -> occursin(key, x), readdir(path))


function getDictofXml(annotationFile::String)
    doc = EzXML.readxml(annotationFile)
    return XMLDict.xml_dict(doc)
end






getDictofXml (generic function with 1 method)

In [11]:
path = "/Users/kaaso/Documents/coding/JuliaEO2024/data/S1A_IW_GRDH_1SDV_20220612T173329_20220612T173354_043633_05359A_EA25.SAFE"

"/Users/kaaso/Documents/coding/JuliaEO2024/data/S1A_IW_GRDH_1SDV_20220612T173329_20220612T173354_043633_05359A_EA25.SAFE"

In [12]:
annotationFolder = getAnnotationPaths("/Users/kaaso/Documents/coding/JuliaEO2024/data/S1A_IW_GRDH_1SDV_20220612T173329_20220612T173354_043633_05359A_EA25.SAFE")
DataFolder = getDataPaths("/Users/kaaso/Documents/coding/JuliaEO2024/data/S1A_IW_GRDH_1SDV_20220612T173329_20220612T173354_043633_05359A_EA25.SAFE")

2-element Vector{String}:
 "/Users/kaaso/Documents/coding/J" ⋯ 139 bytes ⋯ "t173354-043633-05359a-002.tiff"
 "/Users/kaaso/Documents/coding/J" ⋯ 139 bytes ⋯ "t173354-043633-05359a-001.tiff"

In [13]:
meta_dict = getDictofXml(annotationFolder[1])

OrderedCollections.OrderedDict{Any, Any} with 3 entries:
  :version  => "1.0"
  :encoding => "UTF-8"
  "product" => OrderedDict{Any, Any}("adsHeader"=>OrderedDict{Any, Any}("missio…

In [19]:
# Assuming `data` is your 210-element Vector of OrderedDicts
lines = Float64[]
pixels = Float64[]
latitudes = Float64[]
longitudes = Float64[]
# ... similarly for other parameters like height, incidenceAngle, etc.

for item in geo
    push!(lines, parse(Float64, item["line"]))
    push!(pixels, parse(Float64, item["pixel"]))
    push!(latitudes, parse(Float64, item["latitude"]))
    push!(longitudes, parse(Float64, item["longitude"]))
    # ... similarly for other parameters
end

# Now you have arrays `lines`, `pixels`, `latitudes`, `longitudes`, etc.
# You can use these arrays in your interpolation function.


In [22]:
struct SatelliteData
    azimuthTime::String
    slantRangeTime::Float64
    line::Float64
    pixel::Float64
    latitude::Float64
    longitude::Float64
    height::Float64
    incidenceAngle::Float64
    elevationAngle::Float64
end


In [31]:
function extractData(data::Vector{Any})
    satelliteDataArray = SatelliteData[]

    for item in data
        newData = SatelliteData(
            item["azimuthTime"],
            parse(Float64, item["slantRangeTime"]),
            parse(Float64, item["line"]),
            parse(Float64, item["pixel"]),
            parse(Float64, item["latitude"]),
            parse(Float64, item["longitude"]),
            parse(Float64, item["height"]),
            parse(Float64, item["incidenceAngle"]),
            parse(Float64, item["elevationAngle"])
        )
        push!(satelliteDataArray, newData)
    end

    return satelliteDataArray
end


extractData (generic function with 2 methods)

In [27]:
function bilinearInterpolation(x, y, x_values, y_values, z_values)
    # Find the indices of the surrounding points
    x1_idx = findlast(x_values .<= x)
    x2_idx = findfirst(x_values .>= x)
    y1_idx = findlast(y_values .<= y)
    y2_idx = findfirst(y_values .>= y)

    # Handle edge cases
    x1_idx = isnothing(x1_idx) ? 1 : x1_idx
    x2_idx = isnothing(x2_idx) ? length(x_values) : x2_idx
    y1_idx = isnothing(y1_idx) ? 1 : y1_idx
    y2_idx = isnothing(y2_idx) ? length(y_values) : y2_idx

    # Extract the coordinates of the surrounding points
    x1, x2 = x_values[x1_idx], x_values[x2_idx]
    y1, y2 = y_values[y1_idx], y_values[y2_idx]

    # Extract the values at the surrounding points
    Q11 = z_values[y1_idx, x1_idx]
    Q21 = z_values[y1_idx, x2_idx]
    Q12 = z_values[y2_idx, x1_idx]
    Q22 = z_values[y2_idx, x2_idx]

    # Bilinear interpolation formula
    fxy1 = ((x2 - x) / (x2 - x1)) * Q11 + ((x - x1) / (x2 - x1)) * Q21
    fxy2 = ((x2 - x) / (x2 - x1)) * Q12 + ((x - x1) / (x2 - x1)) * Q22
    return ((y2 - y) / (y2 - y1)) * fxy1 + ((y - y1) / (y2 - y1)) * fxy2
end


bilinearInterpolation (generic function with 1 method)

In [33]:
# Assuming `rawData` is your vector of OrderedDicts
satelliteDataArray = extractData(geo);


In [ ]:

# Example: Organizing latitude data for interpolation
lines = [data.line for data in

In [25]:
using OrderedCollections

In [18]:
geo = meta_dict["product"]["geolocationGrid"]["geolocationGridPointList"]["geolocationGridPoint"]

210-element Vector{Any}:
 OrderedCollections.OrderedDict{Any, Any}("azimuthTime" => "2022-06-12T17:33:29.383883", "slantRangeTime" => "5.336815623651522e-03", "line" => "0", "pixel" => "0", "latitude" => "5.094952042895041e+01", "longitude" => "2.051314644505389e+00", "height" => "2.499900186713785e+01", "incidenceAngle" => "3.014398351768650e+01", "elevationAngle" => "2.688594615747057e+01")
 OrderedCollections.OrderedDict{Any, Any}("azimuthTime" => "2022-06-12T17:33:29.383905", "slantRangeTime" => "5.381562410415792e-03", "line" => "0", "pixel" => "1316", "latitude" => "5.097277322756570e+01", "longitude" => "2.235081209024184e+00", "height" => "2.999867081362754e+01", "incidenceAngle" => "3.107068514955396e+01", "elevationAngle" => "2.769412479568636e+01")
 OrderedCollections.OrderedDict{Any, Any}("azimuthTime" => "2022-06-12T17:33:29.383928", "slantRangeTime" => "5.427514046036352e-03", "line" => "0", "pixel" => "2632", "latitude" => "5.099567557892513e+01", "longitude" => "2.41853

In [1]:
te =  Test_name.Sensors.load_tiff(DataFolder[1])

UndefVarError: UndefVarError: `Test_name` not defined

get_coordinate (generic function with 1 method)

In [ ]:

mutable struct Sentinel1GRD <: GroundRangeDetected
    metadata::Sentinel1MetaData
    data::Array{Complex{Float64},2}
end


"""
    get_window(image::Sentinel1SLC)

Returns the window of the complete Sentinel image covered the "image"
"""
function get_window(image::Sentinel1SLC)
    rows_start = image.index_start[1]
    rows_end = size(image.data)[1] + image.index_start[1] - 1

    columns_start = image.index_start[2]
    columns_end = size(image.data)[2] + image.index_start[2] - 1

    return [[rows_start, rows_end],[columns_start, columns_end]]
end


In [18]:
using ArchGDAL

In [6]:
annotationFolder[1]

"/Users/kaaso/Documents/coding/JuliaEO2024/data/S1A_IW_GRDH_1SDV_20220612T173329_20220612T173354_043633_05359A_EA25.SAFE/annotation/s1a-iw-grd-vh-20220612t173329-20220612t173354-043633-05359a-002.xml"

In [3]:
tif = "/Users/kaaso/Documents/coding/JuliaEO2024/data/S1A_IW_GRDH_1SDV_20220612T173329_20220612T173354_043633_05359A_EA25.SAFE/measurement/s1a-iw-grd-vh-20220612t173329-20220612t173354-043633-05359a-002.tiff"

"/Users/kaaso/Documents/coding/JuliaEO2024/data/S1A_IW_GRDH_1SDV_20220612T173329_20220612T173354_043633_05359A_EA25.SAFE/measurement/s1a-iw-grd-vh-20220612t173329-20220612t173354-043633-05359a-002.tiff"

In [8]:
pa = Test_name.Sensors.Sentinel1.getAnnotationPaths("/Users/kaaso/Documents/coding/JuliaEO2024/data/S1A_IW_GRDH_1SDV_20220612T173329_20220612T173354_043633_05359A_EA25.SAFE")
ic1 = Test_name.Sensors.Sentinel1.getDictofXml(pa[1])
meta_dict = Test_name.Sensors.Sentinel1.read_xml_as_dict(pa[1])



OrderedCollections.OrderedDict{Any, Any} with 3 entries:
  :version  => "1.0"
  :encoding => "UTF-8"
  "product" => OrderedDict{Any, Any}("adsHeader"=>OrderedDict{Any, Any}("missio…

In [12]:
hea = Test_name.Sensors.Sentinel1.Sentinel1GRDMetaData(meta_dict);
im = Test_name.Sensors.Sentinel1.Sentinel1ImageInformation(meta_dict);
geo = Test_name.Sensors.Sentinel1.Sentinel1GeolocationGrid(meta_dict);


fieldnames(geo)

In [16]:
fieldnames(Test_name.Sensors.Sentinel1.Sentinel1GeolocationGrid)

(:lines, :samples, :latitude, :longitude, :azimuth_time, :slant_range_time_seconds, :elevation_angle, :incidence_angle, :height)

In [34]:
(geo.samples)

210-element Vector{Int64}:
     1
  1317
  2633
  3949
  5265
  6581
  7897
  9213
 10529
 11845
     ⋮
 15793
 17109
 18425
 19741
 21057
 22373
 23689
 25005
 26316

In [50]:
import Pkg
Pkg.add("Dierckx")

   Resolving package versions...


   Installed Dierckx ───── v0.5.3
   Installed Dierckx_jll ─ v0.1.0+0


    Updating `~/Documents/coding/JuliaEO2024/Project.toml`
  [39dd38d3] + Dierckx v0.5.3
    Updating `~/Documents/coding/JuliaEO2024/Manifest.toml`
 

 [39dd38d3] + Dierckx v0.5.3
  [cd4c43a9] + Dierckx_jll v0.1.0+0


Precompiling 

project...


  ✓ Dierckx_jll

  ✓ Dierckx


  2 dependencies successfully precompiled in 2 seconds. 495 already precompiled. 2 skipped during auto due to previous errors.


In [64]:
using Dierckx

function get_coordinate(
    row, column, lat_gridpoints, long_gridpoints, row_gridpoints, column_gridpoints
)
    """
    Get coordinate from index by interpolating grid-points
    Args:
        row (Integer): index of the row of interest position
        column (Integer): index of the column of interest position
        lat_gridpoints (Vector{Float64}): Latitude of grid-points
        long_gridpoints (Vector{Float64}): Longitude of grid-points
        row_gridpoints (Vector{Int64}): row of grid-points
        column_gridpoints (Vector{Int64}): column of grid-points
    Returns:
        lat (Float64): Latitude of the position
        long (Float64): Longitude of the position
    """

    # Combine row and column gridpoints for interpolation
    points = hcat(row_gridpoints, column_gridpoints)

    # Adjust the smoothing factor 's' as needed
    s = 0.0  # Start with 0 and increase if necessary

    # Create the spline interpolations
    lat_spline = Spline2D(points[:,1], points[:,2], lat_gridpoints, s=s)
    long_spline = Spline2D(points[:,1], points[:,2], long_gridpoints, s=s)

    # Interpolate to get lat and long
    lat = evaluate(lat_spline, row, column)
    long = evaluate(long_spline, row, column)

    return lat, long
end

# Example usage
# Replace these with your actual data
lat_values = geo.longitude; # Vector{Float64} with latitude values
long_values = geo.latitude; # Vector{Float64} with longitude values
row_gridpoints = geo.lines; # Vector{Int64} representing row grid points
column_gridpoints = geo.samples; # Vector{Int64} representing column grid points



In [65]:

# Get lat, lon for a specific row and column
lat, long = get_coordinate1(100, 150, lat_values, long_values, row_gridpoints, column_gridpoints)

ErrorException: The required storage space exceeds the available storage space:
nxest or nyest too small, or s too small. Try increasing s.

In [63]:
using Interpolations

function get_coordinate(row, column, lat_values, long_values, row_gridpoints, column_gridpoints)
    # Convert row and column grid points to Float64
    row_gridpoints_float = Float64.(row_gridpoints)
    column_gridpoints_float = Float64.(column_gridpoints)

    # Create cubic spline interpolators
    lat_interp = CubicSplineInterpolation(row_gridpoints_float, lat_values, extrapolation_bc=Flat())
    long_interp = CubicSplineInterpolation(column_gridpoints_float, long_values, extrapolation_bc=Flat())

    # Interpolate latitude and longitude
    lat = lat_interp(row)
    long = long_interp(column)

    return lat, long
end



lat, long = get_coordinate(100, 150, lat_values, long_values, row_gridpoints, column_gridpoints)


UndefVarError: UndefVarError: `Flat` not defined

In [29]:
get_coordinate3(1000,1000, geo.latitude, geo.longitude, geo.lines, geo.samples)

MethodError: MethodError: no method matching interpolate(::Tuple{Vector{Float64}, Vector{Float64}}, ::Vector{Float64}, ::Gridded{Linear{Throw{OnGrid}}})

Closest candidates are:
  interpolate(::Tuple{Vararg{Union{AbstractVector{T}, Tuple} where T, N}}, !Matched::AbstractArray{Tel, N}, ::IT) where {Tel, N, IT<:Union{NoInterp, Tuple{Vararg{Union{NoInterp, Gridded}}}, Gridded}}
   @ Interpolations ~/.julia/packages/Interpolations/nDwIa/src/gridded/gridded.jl:165
  interpolate(!Matched::Type{TWeights}, !Matched::Type{TC}, ::Any, !Matched::IT) where {TWeights, TC, IT<:Union{NoInterp, Tuple{Vararg{Union{NoInterp, BSpline}}}, BSpline}}
   @ Interpolations ~/.julia/packages/Interpolations/nDwIa/src/b-splines/b-splines.jl:164
  interpolate(!Matched::Type{TWeights}, !Matched::Type{TC}, ::Any, !Matched::IT, !Matched::Real, !Matched::Int64) where {TWeights, TC, IT<:Union{NoInterp, Tuple{Vararg{Union{NoInterp, BSpline}}}, BSpline}}
   @ Interpolations ~/.julia/packages/Interpolations/nDwIa/src/b-sp

In [56]:
using Interpolations

function get_coordinatef(
    row, column, lat_gridpoints, long_gridpoints, row_gridpoints, column_gridpoints
)
    """
    Get coordinate from index by interpolating grid-points
    Args:
        row (Integer): index of the row of interest position
        column (Integer): index of the column of interest position
        lat_gridpoints (Vector{Float64}): Latitude of grid-points
        long_gridpoints (Vector{Float64}): Longitude of grid-points
        row_gridpoints (Vector{Int64}): row of grid-points
        column_gridpoints (Vector{Int64}): column of grid-points
    Returns:
        lat (Float64): Latitude of the position
        long (Float64): Longitude of the position
    """

    # Assuming regular grid for BSpline interpolation
    lat_itp = interpolate((row_gridpoints, column_gridpoints), lat_gridpoints, Gridded(Linear()))
    long_itp = interpolate((row_gridpoints, column_gridpoints), long_gridpoints, Gridded(Linear()))

    lat = lat_itp[row, column]
    long = long_itp[row, column]

    return lat, long
end


get_coordinatef (generic function with 1 method)

In [58]:
get_coordinatef(1000,1000, geo.latitude, geo.longitude, geo.lines, geo.samples)

MethodError: MethodError: no method matching interpolate(::Tuple{Vector{Int64}, Vector{Int64}}, ::Vector{Float64}, ::Gridded{Linear{Throw{OnGrid}}})

Closest candidates are:
  interpolate(::Tuple{Vararg{Union{AbstractVector{T}, Tuple} where T, N}}, !Matched::AbstractArray{Tel, N}, ::IT) where {Tel, N, IT<:Union{NoInterp, Tuple{Vararg{Union{NoInterp, Gridded}}}, Gridded}}
   @ Interpolations ~/.julia/packages/Interpolations/nDwIa/src/gridded/gridded.jl:165
  interpolate(!Matched::Type{TWeights}, !Matched::Type{TC}, ::Any, !Matched::IT) where {TWeights, TC, IT<:Union{NoInterp, Tuple{Vararg{Union{NoInterp, BSpline}}}, BSpline}}
   @ Interpolations ~/.julia/packages/Interpolations/nDwIa/src/b-splines/b-splines.jl:164
  interpolate(!Matched::Type{TWeights}, !Matched::Type{TC}, ::Any, !Matched::IT, !Matched::Real, !Matched::Int64) where {TWeights, TC, IT<:Union{NoInterp, Tuple{Vararg{Union{NoInterp, BSpline}}}, BSpline}}
   @ Interpolations ~/.julia/packages/Interpolations/nDwIa/src/b-spline

In [28]:
using Interpolations
using Optim
using LinearAlgebra

function get_coordinate2(row, column, lat_gridpoints, long_gridpoints, row_gridpoints, column_gridpoints)
    # Interpolation for latitude and longitude
    lat_interp = interpolate((row_gridpoints, column_gridpoints), lat_gridpoints, Gridded(Linear()))
    long_interp = interpolate((row_gridpoints, column_gridpoints), long_gridpoints, Gridded(Linear()))

    lat = lat_interp[row, column]
    long = long_interp[row, column]

    return lat, long
end


function get_coordinate3(row, column, lat_gridpoints, long_gridpoints, row_gridpoints, column_gridpoints)
    # Convert row and column grid points to Float64
    row_gridpoints_float = float(row_gridpoints)
    column_gridpoints_float = float(column_gridpoints)

    # Interpolation for latitude and longitude
    lat_interp = interpolate((row_gridpoints_float, column_gridpoints_float), lat_gridpoints, Gridded(Linear()))
    long_interp = interpolate((row_gridpoints_float, column_gridpoints_float), long_gridpoints, Gridded(Linear()))

    lat = lat_interp[row, column]
    long = long_interp[row, column]

    return lat, long
end


get_coordinate3 (generic function with 1 method)

In [10]:
reference_time = Test_name.Sensors.Sentinel1.get_reference_time(meta_dict)

2022-06-12T17:33:29

In [19]:
coordinateConversion2= meta_dict["product"]["coordinateConversion"]["coordinateConversionList"]["coordinateConversion"];

azimuthTime = [Test_name.Sensors.Sentinel1.parse_delta_time(elem["azimuthTime"],reference_time) for elem in coordinateConversion2];
grsrCoefficients = [parse.(Float64,split(elem["grsrCoefficients"][""]," ")) for elem in coordinateConversion2]



28-element Vector{Vector{Float64}}:
 [799968.5368545058, 0.5027890283195999, 5.346541292395387e-7, -3.380811179770082e-13, 3.213496265378736e-20, 2.088330129077014e-25, -2.44654383152174e-31, 1.090090486279145e-37, 2.149707122104422e-45]
 [799968.5368545763, 0.5027524123846079, 5.346785975589008e-7, -3.380721747596121e-13, 3.210272538043584e-20, 2.088156094212409e-25, -2.444191438793631e-31, 1.084353092800081e-37, 2.680360963399394e-45]
 [799968.5368536464, 0.502700560495108, 5.347133561801165e-7, -3.380580359717493e-13, 3.203193497445918e-20, 2.090028148148028e-25, -2.450556741171178e-31, 1.098993862788715e-37, 1.284073155506245e-45]
 [799968.5368535054, 0.5026536074124773, 5.347448083080476e-7, -3.380456266609052e-13, 3.197421474797523e-20, 2.091172324639931e-25, -2.453750574063504e-31, 1.106244897336468e-37, 5.717415818159479e-46]
 [799968.5368541314, 0.5026183876459421, 5.347683296450658e-7, -3.380371433031531e-13, 3.194483740158948e-20, 2.090888434645832e-25, -2.450935125154188e-3

In [18]:
typeof(gr0)

Vector{Float64} (alias for Array{Float64, 1})

In [20]:
typeof(grsrCoefficients)

Vector{Vector{Float64}} (alias for Array{Array{Float64, 1}, 1})

In [ ]:
geo.lines, geo.samples

In [68]:
geo.samples[1]

1

In [69]:
geo.samples[end]

26316

In [74]:
x = range(geo.samples[1], geo.samples[end], length=geo.samples[end])
y = range(geo.lines[1], geo.lines[end], length=geo.lines[end])

1.0:1.0:16683.0

In [75]:
itp = LinearInterpolation((x, y),geo.latitude)


MethodError: MethodError: no method matching linear_interpolation(::Tuple{StepRangeLen{Float64, Base.TwicePrecision{Float64}, Base.TwicePrecision{Float64}, Int64}, StepRangeLen{Float64, Base.TwicePrecision{Float64}, Base.TwicePrecision{Float64}, Int64}}, ::Vector{Float64})

Closest candidates are:
  linear_interpolation(!Matched::AbstractRange, ::AbstractVector; extrapolation_bc)
   @ Interpolations ~/.julia/packages/Interpolations/nDwIa/src/convenience-constructors.jl:7
  linear_interpolation(!Matched::AbstractVector, ::AbstractVector; extrapolation_bc)
   @ Interpolations ~/.julia/packages/Interpolations/nDwIa/src/convenience-constructors.jl:9
  linear_interpolation(::Tuple{Vararg{AbstractRange, N}}, !Matched::AbstractArray{T, N}; extrapolation_bc) where {N, T}
   @ Interpolations ~/.julia/packages/Interpolations/nDwIa/src/convenience-constructors.jl:23
  ...


interpolateLatLon (generic function with 1 method)

In [117]:
interpolateLatLon(1000,1000, geo.lines, geo.samples, geo.latitude, geo.longitude)

(51.12648977863641, 3.5435370018551167)

In [119]:
lat_matrix, lon_matrix = interpolateLatLonGrid(geo.lines, geo.samples, geo.latitude, geo.longitude)


([50.94952042895041 50.949687068910464 … 52.68383922318156 52.6840057278105; 50.949532043734926 50.94969868258563 … 52.683849281873705 52.68401578533434; … ; 51.14816423965925 51.14831092108828 … 52.85377852563578 52.85392375370793; 51.1481956160664 51.1483422941445 … 52.85380491679139 52.8539501412217], [2.051314644505389 2.052715957067042 … 3.6511129819207775 3.652568212384678; 2.051406435996158 2.0528077491704444 … 3.651208786002854 3.6526640172020017; … ; 3.7099485386992113 3.7113616987696942 … 5.38238749971355 5.383854469490045; 3.710226139392344 3.7116393013379083 … 5.3826763860687885 5.384143356939436])

In [130]:
cord = interpolateLatLonForPoints([10,20,30,50,100], [10,20,30,50,100], geo.lines, geo.samples, geo.latitude, geo.longitude)

Dict{String, Vector} with 4 entries:
  "column" => [10, 20, 30, 50, 100]
  "rows"   => [10, 20, 30, 50, 100]
  "lats"   => [50.9511, 50.9529, 50.9547, 50.9583, 50.9672]
  "Long"   => [2.06475, 2.07968, 2.09462, 2.12448, 2.19914]

In [120]:
size(lat_matrix)

(16683, 26316)

In [159]:
row_test = [1:10]
column_test = [1:1000]

TypeError: TypeError: in typeassert, expected Type, got a value of type Int64

1.0:1.0:100.0

In [151]:
size([1,2,3,4,5,6])

(6,)

In [173]:
using BenchmarkTools

In [182]:
@benchmark s1 = interpolateLatLonForPoints2(range(1,20000,20000), range(1,20000,20000),geo.lines, geo.samples, geo.latitude, geo.longitude, geo.incidence_angle, geo.height, geo.azimuth_time, geo.slant_range_time_seconds, geo.elevation_angle)

BoundsError: BoundsError: attempt to access 10×21 interpolate((::Vector{Int64},::Vector{Int64}), ::Matrix{Float64}, Gridded(Linear())) with element type Float64 at index [16684.0, 16684.0]

In [181]:
@benchmark s2 = interpolateLatLonForPoints4(range(1,20000,20000), range(1,20000,20000),geo.lines, geo.samples, geo.latitude, geo.longitude, geo.incidence_angle, geo.height, geo.azimuth_time, geo.slant_range_time_seconds, geo.elevation_angle)

BenchmarkTools.Trial: 89 samples with 1 evaluation.
 Range (min … max):  48.970 ms … 73.087 ms  ┊ GC (min … max): 0.00% … 30.00%
 Time  (median):     52.259 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   56.925 ms ±  8.395 ms  ┊ GC (mean ± σ):  9.84% ± 12.86%

        █ ▂                                                    
  ▄▃▄▄▄▇███▄▆▄▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▅▄▃▃▄▃▄▆▃▄▁▃ ▁
  49 ms           Histogram: frequency by time        72.8 ms <

 Memory estimate: 71.21 MiB, allocs estimate: 40094.

In [179]:
using Interpolations

function interpolateLatLonForPoints(wanted_rows, wanted_columns, rows, columns, lats, lons, incidenceangle, height, azimuth_time, slant_range_time_seconds, elevation_angle)
    """
    Efficiently interpolate the latitude, longitude, incidence angle, height, azimuth time, slant range time, and elevation angle for given vectors of rows and columns using nearest values.

    Parameters:
    - wanted_rows: Vector of wanted row numbers.
    - wanted_columns: Vector of wanted column numbers.
    - rows: Array of row numbers.
    - columns: Array of column numbers.
    - lats: Array of latitudes.
    - lons: Array of longitudes.
    - incidenceangle: Array of incidence angles.
    - height: Array of heights.
    - azimuth_time: Array of azimuth times.
    - slant_range_time_seconds: Array of slant range times in seconds.
    - elevation_angle: Array of elevation angles.

    Returns:
    - Dictionary containing vectors for interpolated latitudes, longitudes, incidence angles, heights, azimuth times, slant range times, and elevation angles.
    """

    # Function to find the nearest points
    function find_nearest(array, value)
        idx = argmin(abs.(array .- value))
        return array[idx]
    end

    # Initialize vectors for the interpolated values
    interpolated_lats = Float64[]
    interpolated_lons = Float64[]
    interpolated_incidence = Float64[]
    interpolated_height = Float64[]
    interpolated_azimuth = Float64[]
    interpolated_slant_range = Float64[]
    interpolated_elevation = Float64[]

    # Convert the zipped iterator to an array of tuples
    row_col_pairs = collect(zip(rows, columns))

    # Iterate over each wanted row and column
    for (wanted_row, wanted_column) in zip(wanted_rows, wanted_columns)
        # Find the nearest row and column
        nearest_row = find_nearest(rows, wanted_row)
        nearest_col = find_nearest(columns, wanted_column)

        # Find the index of the nearest row and column
        idx = findfirst(isequal((nearest_row, nearest_col)), row_col_pairs)

        # Interpolate the values
        push!(interpolated_lats, lats[idx])
        push!(interpolated_lons, lons[idx])
        push!(interpolated_incidence, incidenceangle[idx])
        push!(interpolated_height, height[idx])
        push!(interpolated_azimuth, azimuth_time[idx])
        push!(interpolated_slant_range, slant_range_time_seconds[idx])
        push!(interpolated_elevation, elevation_angle[idx])
    end
    res = Dict("rows" => wanted_rows, 
            "columns"=> wanted_columns,
            "latitudes" => interpolated_lats,
            "longitudes" => interpolated_lons,
            "incidence_angles" => interpolated_incidence,
            "heights" => interpolated_height,
            "azimuth_times" => interpolated_azimuth,
            "slant_range_times" => interpolated_slant_range,
            "elevation_angles" => interpolated_elevation)


    # Return the results in a dictionary
    #return Dict("rows" => wanted_rows, "columns" => wanted_columns, "lats" => interpolated_lats, "Long" => interpolated_lons, "incidence_angles" => interpolated_incidence, "heights" => interpolated_height, "azimuth_times" => interpolated_azimuth, "slant_range_times" => interpolated_slant_range, "elevation_angles" => interpolated_elevation)
    return res
end


interpolateLatLonForPoints4 (generic function with 1 method)

In [180]:
using Interpolations

function interpolateLatLonForPoints2(wanted_rows, wanted_columns, rows, columns, lats, lons, incidenceangle, height, azimuth_time, slant_range_time_seconds, elevation_angle)
    """
    Interpolate the latitude, longitude, incidence angle, height, azimuth time, slant range time, and elevation angle for given vectors of rows and columns.

    Parameters:
    - wanted_rows: Vector of wanted row numbers.
    - wanted_columns: Vector of wanted column numbers.
    - rows: Array of row numbers.
    - columns: Array of column numbers.
    - lats: Array of latitudes.
    - lons: Array of longitudes.
    - incidenceangle: Array of incidence angles.
    - height: Array of heights.
    - azimuth_time: Array of azimuth times.
    - slant_range_time_seconds: Array of slant range times in seconds.
    - elevation_angle: Array of elevation angles.

    Returns:
    - Vectors for interpolated latitudes, longitudes, incidence angles, heights, azimuth times, slant range times, and elevation angles.
    """

    # Create a grid for interpolation
    row_grid = unique(rows)
    col_grid = unique(columns)
    lat_grid = reshape(lats, length(row_grid), length(col_grid))
    lon_grid = reshape(lons, length(row_grid), length(col_grid))
    incidence_grid = reshape(incidenceangle, length(row_grid), length(col_grid))
    height_grid = reshape(height, length(row_grid), length(col_grid))
    azimuth_grid = reshape(azimuth_time, length(row_grid), length(col_grid))
    slant_range_grid = reshape(slant_range_time_seconds, length(row_grid), length(col_grid))
    elevation_grid = reshape(elevation_angle, length(row_grid), length(col_grid))

    # Interpolation functions
    lat_interp = interpolate((row_grid, col_grid), lat_grid, Gridded(Linear()))
    lon_interp = interpolate((row_grid, col_grid), lon_grid, Gridded(Linear()))
    incidence_interp = interpolate((row_grid, col_grid), incidence_grid, Gridded(Linear()))
    height_interp = interpolate((row_grid, col_grid), height_grid, Gridded(Linear()))
    azimuth_interp = interpolate((row_grid, col_grid), azimuth_grid, Gridded(Linear()))
    slant_range_interp = interpolate((row_grid, col_grid), slant_range_grid, Gridded(Linear()))
    elevation_interp = interpolate((row_grid, col_grid), elevation_grid, Gridded(Linear()))

    # Initialize vectors for the interpolated values
    interpolated_lats = Float64[]
    interpolated_lons = Float64[]
    interpolated_incidence = Float64[]
    interpolated_height = Float64[]
    interpolated_azimuth = Float64[]
    interpolated_slant_range = Float64[]
    interpolated_elevation = Float64[]

    # Interpolate for each pair of wanted rows and columns
    for (row, col) in zip(wanted_rows, wanted_columns)
        push!(interpolated_lats, lat_interp(row, col))
        push!(interpolated_lons, lon_interp(row, col))
        push!(interpolated_incidence, incidence_interp(row, col))
        push!(interpolated_height, height_interp(row, col))
        push!(interpolated_azimuth, azimuth_interp(row, col))
        push!(interpolated_slant_range, slant_range_interp(row, col))
        push!(interpolated_elevation, elevation_interp(row, col))
    end

    res = Dict("rows" => wanted_rows, 
            "columns"=> wanted_columns,
            "latitudes" => interpolated_lats,
            "longitudes" => interpolated_lons,
            "incidence_angles" => interpolated_incidence,
            "heights" => interpolated_height,
            "azimuth_times" => interpolated_azimuth,
            "slant_range_times" => interpolated_slant_range,
            "elevation_angles" => interpolated_elevation)

    #nterpolated_lats, interpolated_lons, interpolated_incidence, interpolated_height, interpolated_azimuth, interpolated_slant_range, interpolated_elevation
    return res
end


interpolateLatLonForPoints2 (generic function with 1 method)

In [118]:
using Interpolations

using Interpolations

function interpolateLatLonForPoints(wanted_rows, wanted_columns, rows, columns, lats, lons)
    """
    Interpolate the latitude and longitude for given vectors of rows and columns.

    Parameters:
    - wanted_rows: Vector of wanted row numbers.
    - wanted_columns: Vector of wanted column numbers.
    - rows: Array of row numbers.
    - columns: Array of column numbers.
    - lats: Array of latitudes.
    - lons: Array of longitudes.

    Returns:
    - Two vectors: one for interpolated latitudes and one for interpolated longitudes.
    """

    # Create a grid for interpolation
    row_grid = unique(rows)
    col_grid = unique(columns)
    lat_grid = reshape(lats, length(row_grid), length(col_grid))
    lon_grid = reshape(lons, length(row_grid), length(col_grid))

    # Interpolation functions
    lat_interp = interpolate((row_grid, col_grid), lat_grid, Gridded(Linear()))
    lon_interp = interpolate((row_grid, col_grid), lon_grid, Gridded(Linear()))

    # Initialize vectors for latitudes and longitudes
    interpolated_lats = Float64[]
    interpolated_lons = Float64[]

    # Interpolate for each pair of wanted rows and columns
    for (row, col) in zip(wanted_rows, wanted_columns)
        push!(interpolated_lats, lat_interp(row, col))
        push!(interpolated_lons, lon_interp(row, col))
    end

    return Dict("rows" =>wanted_rows, "column"=>wanted_columns,   "lats" => interpolated_lats, "Long" => interpolated_lons)
end




function interpolateLatLon(wanted_row, wanted_column, rows, columns, lats, lons)
    """
    Interpolate the latitude and longitude for a given row and column.

    Parameters:
    - wanted_row: The row number to find.
    - wanted_column: The column number to find.
    - rows: Array of row numbers.
    - columns: Array of column numbers.
    - lats: Array of latitudes.
    - lons: Array of longitudes.

    Returns:
    - (latitude, longitude) interpolated values.
    """

    # Create a grid for interpolation
    row_grid = unique(rows)
    col_grid = unique(columns)
    lat_grid = reshape(lats, length(row_grid), length(col_grid))
    lon_grid = reshape(lons, length(row_grid), length(col_grid))

    # Interpolation functions
    lat_interp = interpolate((row_grid, col_grid), lat_grid, Gridded(Linear()))
    lon_interp = interpolate((row_grid, col_grid), lon_grid, Gridded(Linear()))

    # Interpolate latitude and longitude
    interpolated_lat = lat_interp(wanted_row, wanted_column)
    interpolated_lon = lon_interp(wanted_row, wanted_column)

    return interpolated_lat, interpolated_lon
end



function interpolateLatLonGrid(rows, columns, lats, lons)
    """
    Interpolate the latitude and longitude for the entire grid.

    Parameters:
    - rows: Array of row numbers.
    - columns: Array of column numbers.
    - lats: Array of latitudes.
    - lons: Array of longitudes.

    Returns:
    - Two matrices: one for interpolated latitudes and one for interpolated longitudes.
    """

    # Create a grid for interpolation
    row_grid = unique(rows)
    col_grid = unique(columns)
    lat_grid = reshape(lats, length(row_grid), length(col_grid))
    lon_grid = reshape(lons, length(row_grid), length(col_grid))

    # Interpolation functions
    lat_interp = interpolate((row_grid, col_grid), lat_grid, Gridded(Linear()))
    lon_interp = interpolate((row_grid, col_grid), lon_grid, Gridded(Linear()))

    # Define the full grid for interpolation
    full_row_grid = minimum(row_grid):maximum(row_grid)
    full_col_grid = minimum(col_grid):maximum(col_grid)

    # Initialize matrices for latitudes and longitudes
    lat_matrix = Array{Float64}(undef, length(full_row_grid), length(full_col_grid))
    lon_matrix = Array{Float64}(undef, length(full_row_grid), length(full_col_grid))

    # Populate the matrices
    for (i, row) in enumerate(full_row_grid)
        for (j, col) in enumerate(full_col_grid)
            lat_matrix[i, j] = lat_interp(row, col)
            lon_matrix[i, j] = lon_interp(row, col)
        end
    end

    return lat_matrix, lon_matrix
end

interpolateLatLonGrid (generic function with 1 method)

signalet er det samme. Stoejen er voksen. 

hvor meget daarlgiere signalet er, kan svare til afstanden til hvor den er.

In [ ]:
itp = LinearInterpolation((x, y), z)

In [109]:
x = range(geo.samples[1], geo.samples[end], length=geo.samples[end])
y = range(geo.lines[1], geo.lines[end], length=geo.lines[end])

1.0:1.0:16683.0

In [110]:
column_gridpoints = (range(geo.samples[1], geo.samples[end], length=geo.samples[end]))
row_gridpoints = (range(geo.lines[1], geo.lines[end], length=geo.lines[end]))

lat_interp = CubicSplineInterpolation((x, y), extrapolation_bc=Line())

MethodError: MethodError: no method matching cubic_spline_interpolation(::Tuple{StepRangeLen{Float64, Base.TwicePrecision{Float64}, Base.TwicePrecision{Float64}, Int64}, StepRangeLen{Float64, Base.TwicePrecision{Float64}, Base.TwicePrecision{Float64}, Int64}}; extrapolation_bc::Line{Nothing})

Closest candidates are:
  cubic_spline_interpolation(::Tuple{Vararg{AbstractRange, N}}, !Matched::AbstractArray{T, N}; bc, extrapolation_bc) where {N, T}
   @ Interpolations ~/.julia/packages/Interpolations/nDwIa/src/convenience-constructors.jl:29
  cubic_spline_interpolation(!Matched::AbstractRange, !Matched::AbstractVector; bc, extrapolation_bc)
   @ Interpolations ~/.julia/packages/Interpolations/nDwIa/src/convenience-constructors.jl:12


In [ ]:
geo.lines, geo.samples

16683-element Vector{Int64}:
     1
     2
     3
     4
     5
     6
     7
     8
     9
    10
     ⋮
 16675
 16676
 16677
 16678
 16679
 16680
 16681
 16682
 16683

In [90]:
get_coordinate(1000,1000, geo.latitude, geo.longitude, geo.lines, geo.samples)

ArgumentError: ArgumentError: The range 1.0:1.0:16683.0 is incommensurate with the corresponding axis Base.OneTo(210)

In [80]:
using Interpolations, Plots
# Data
vector1 = range(geo.samples[1], geo.samples[end], length=geo.samples[end])
vector2 = range(geo.lines[1], geo.lines[end], length=geo.lines[end])
grid = [(x, y) for x in vector1, y in vector2]

26316×16683 Matrix{Tuple{Float64, Float64}}:
 (1.0, 1.0)      (1.0, 2.0)      (1.0, 3.0)      …  (1.0, 16683.0)
 (2.0, 1.0)      (2.0, 2.0)      (2.0, 3.0)         (2.0, 16683.0)
 (3.0, 1.0)      (3.0, 2.0)      (3.0, 3.0)         (3.0, 16683.0)
 (4.0, 1.0)      (4.0, 2.0)      (4.0, 3.0)         (4.0, 16683.0)
 (5.0, 1.0)      (5.0, 2.0)      (5.0, 3.0)         (5.0, 16683.0)
 (6.0, 1.0)      (6.0, 2.0)      (6.0, 3.0)      …  (6.0, 16683.0)
 (7.0, 1.0)      (7.0, 2.0)      (7.0, 3.0)         (7.0, 16683.0)
 (8.0, 1.0)      (8.0, 2.0)      (8.0, 3.0)         (8.0, 16683.0)
 (9.0, 1.0)      (9.0, 2.0)      (9.0, 3.0)         (9.0, 16683.0)
 (10.0, 1.0)     (10.0, 2.0)     (10.0, 3.0)        (10.0, 16683.0)
 ⋮                                               ⋱  
 (26308.0, 1.0)  (26308.0, 2.0)  (26308.0, 3.0)     (26308.0, 16683.0)
 (26309.0, 1.0)  (26309.0, 2.0)  (26309.0, 3.0)     (26309.0, 16683.0)
 (26310.0, 1.0)  (26310.0, 2.0)  (26310.0, 3.0)     (26310.0, 16683.0)
 (26311.0, 1.0)  (

In [81]:
grid

26316×16683 Matrix{Tuple{Float64, Float64}}:
 (1.0, 1.0)      (1.0, 2.0)      (1.0, 3.0)      …  (1.0, 16683.0)
 (2.0, 1.0)      (2.0, 2.0)      (2.0, 3.0)         (2.0, 16683.0)
 (3.0, 1.0)      (3.0, 2.0)      (3.0, 3.0)         (3.0, 16683.0)
 (4.0, 1.0)      (4.0, 2.0)      (4.0, 3.0)         (4.0, 16683.0)
 (5.0, 1.0)      (5.0, 2.0)      (5.0, 3.0)         (5.0, 16683.0)
 (6.0, 1.0)      (6.0, 2.0)      (6.0, 3.0)      …  (6.0, 16683.0)
 (7.0, 1.0)      (7.0, 2.0)      (7.0, 3.0)         (7.0, 16683.0)
 (8.0, 1.0)      (8.0, 2.0)      (8.0, 3.0)         (8.0, 16683.0)
 (9.0, 1.0)      (9.0, 2.0)      (9.0, 3.0)         (9.0, 16683.0)
 (10.0, 1.0)     (10.0, 2.0)     (10.0, 3.0)        (10.0, 16683.0)
 ⋮                                               ⋱  
 (26308.0, 1.0)  (26308.0, 2.0)  (26308.0, 3.0)     (26308.0, 16683.0)
 (26309.0, 1.0)  (26309.0, 2.0)  (26309.0, 3.0)     (26309.0, 16683.0)
 (26310.0, 1.0)  (26310.0, 2.0)  (26310.0, 3.0)     (26310.0, 16683.0)
 (26311.0, 1.0)  (

In [66]:

# Interpolation object (caches coefficients and such)
itp = LinearInterpolation((x, y), z)
# Fine grid
x2 = range(extrema(x)..., length=300)
y2 = range(extrema(y)..., length=200)
# Interpolate
z2 = [itp(x,y) for y in y2, x in x2]

200×300 Matrix{Float64}:
 -0.275027  -0.259086  -0.243145  …  -0.839876  -0.844374  -0.848872
 -0.280031  -0.26409   -0.248149     -0.844879  -0.849378  -0.853876
 -0.285035  -0.269094  -0.253152     -0.849883  -0.854382  -0.85888
 -0.290039  -0.274098  -0.258156     -0.854887  -0.859386  -0.863884
 -0.295043  -0.279102  -0.26316      -0.859891  -0.86439   -0.868888
 -0.300047  -0.284106  -0.268164  …  -0.864895  -0.869394  -0.873892
 -0.305051  -0.289109  -0.273168     -0.869899  -0.874398  -0.878896
 -0.310055  -0.294113  -0.278172     -0.874903  -0.879402  -0.8839
 -0.315059  -0.299117  -0.283176     -0.879907  -0.884406  -0.888904
 -0.320062  -0.304121  -0.28818      -0.884911  -0.88941   -0.893908
  ⋮                               ⋱                        
 -1.14504   -1.1291    -1.11316      -1.70989   -1.71438   -1.71888
 -1.14853   -1.13259   -1.11664      -1.71338   -1.71787   -1.72237
 -1.15202   -1.13607   -1.12013      -1.71686   -1.72136   -1.72586
 -1.1555    -1.13956   -

In [56]:
srgrCoefficients = [parse.(Float64,split(elem["grsrCoefficients"][""]," ")) for elem in coordinateConversion2]

28-element Vector{Vector{Float64}}:
 [799968.5368545058, 0.5027890283195999, 5.346541292395387e-7, -3.380811179770082e-13, 3.213496265378736e-20, 2.088330129077014e-25, -2.44654383152174e-31, 1.090090486279145e-37, 2.149707122104422e-45]
 [799968.5368545763, 0.5027524123846079, 5.346785975589008e-7, -3.380721747596121e-13, 3.210272538043584e-20, 2.088156094212409e-25, -2.444191438793631e-31, 1.084353092800081e-37, 2.680360963399394e-45]
 [799968.5368536464, 0.502700560495108, 5.347133561801165e-7, -3.380580359717493e-13, 3.203193497445918e-20, 2.090028148148028e-25, -2.450556741171178e-31, 1.098993862788715e-37, 1.284073155506245e-45]
 [799968.5368535054, 0.5026536074124773, 5.347448083080476e-7, -3.380456266609052e-13, 3.197421474797523e-20, 2.091172324639931e-25, -2.453750574063504e-31, 1.106244897336468e-37, 5.717415818159479e-46]
 [799968.5368541314, 0.5026183876459421, 5.347683296450658e-7, -3.380371433031531e-13, 3.194483740158948e-20, 2.090888434645832e-25, -2.450935125154188e-3

In [51]:
coordinateConversion[1]

MethodError: MethodError: Cannot `convert` an object of type Int64 to an object of type coordinateConversion
Closest candidates are:
  convert(::Type{T}, !Matched::T) where T at Base.jl:61
  coordinateConversion(::Any, !Matched::Any, !Matched::Any, !Matched::Any, !Matched::Any, !Matched::Any) at ~/Documents/coding/JuliaEO2024/metadata.ipynb:7

MethodError: MethodError: Cannot `convert` an object of type Float64 to an object of type DateTime
Closest candidates are:
  convert(::Type{DateTime}, !Matched::Date) at /Applications/Julia-1.8.app/Contents/Resources/julia/share/julia/stdlib/v1.8/Dates/src/conversions.jl:30
  convert(::Type{DateTime}, !Matched::Millisecond) at /Applications/Julia-1.8.app/Contents/Resources/julia/share/julia/stdlib/v1.8/Dates/src/conversions.jl:34
  convert(::Type{DateTime}, !Matched::PyCall.PyObject) at ~/.julia/packages/PyCall/KLzIO/src/pydates.jl:134
  ...

In [35]:
"""
    Sentinel1MetaData

Metadata structure for the Sentinel-1 satellite for each burst in the swath.

General metadata info is kept in the following structures:
- `Sentinel1Header`
- `Sentinel1ProductInformation`
- `Sentinel1ImageInformation`
- `Sentinel1SwathTiming`
- `Sentinel1GeolocationGrid`
`Sentinel1BurstInformation` specific Info is kept in
- `Vector{Sentinel1BurstInformation}`

# Example
    slcMetadata = Sentinel1MetaData(meta_dict)

# Input
- `meta_dict`: xml file.

Can be accessed as, e.g.,
    slcMetadata.product.radar_frequency --> 5.40500045433435e9::Float64
    slcMetadata.header.swath --> 1::Int
    slcMetadata.header.mode --> "IW"::String
    slcMetadata.header.polarisation --> "VH"::String
"""
Base.@kwdef struct Sentinel1MetaData <: MetaData
    reference_time::DateTime
    header::Sentinel1Header
    image::Sentinel1ImageInformation
    geolocation::Sentinel1GeolocationGrid
end

OrderedCollections.OrderedDict{Any, Any} with 6 entries:
  "azimuthTime"      => "2022-06-12T17:33:27.474464"
  "slantRangeTime"   => "5.336815623657255e-03"
  "sr0"              => "7.999685368545058e+05"
  "srgrCoefficients" => OrderedCollections.OrderedDict{Any, Any}(:count=>"9", "…
  "gr0"              => "0.000000000000000e+00"
  "grsrCoefficients" => OrderedCollections.OrderedDict{Any, Any}(:count=>"9", "…

In [34]:
parse.(Float64,split(s[1]["srgrCoefficients"][""]," "))

9-element Vector{Float64}:
  0.04505718871951103
  1.988863625575908
 -4.199506089919315e-6
  2.261506126701918e-11
 -1.341729268085677e-16
  7.246614758959067e-22
 -2.967740346682937e-27
  7.628800092252304e-33
 -8.924404188174125e-39

"4.505718871951103e-02 1.988863625575908e+00 -4.199506089919315e-06 2.261506126701918e-11 -1.341729268085677e-16 7.246614758959067e-22 -2.967740346682937e-27 7.628800092252304e-33 -8.924404188174125e-39"

In [ ]:
[parse(Float64, elem["slantRangeTime"]) for elem in s]

# Sentinel1ProductInformation(meta_dict)

In [ ]:
"""
    Sentinel1ProductInformation

returns structure of product information
"""
Base.@kwdef struct Sentinel1ProductInformation
    pass::String
    timeliness_category::String
    platform_heading::Float64
    projection::String
    range_sampling_rate::Float64
    radar_frequency::Float64
    azimuth_steering_rate::Float64
end


In [ ]:
""""
    Sentinel1ProductInformation

Constructors for the Sentinel1ProductInformation structure.

It takes a dictionary containing the full sentinel-1 swath metadata and extracts the Sentinel1ProductInformation as a structure. Sentinel1ProductInformation file:
- `pass`: Direction of the orbit (ascending, descending)
- `timeliness_category`: Timeliness category under which the product was produced, i.e. time frame from the data acquisition
- `platform_heading`: Platform heading relative to North [degrees].
- `projection`: Projection of the image, either slant range or ground range.
- `range_sampling_rate`: Range sample rate [Hz]
- `radar_frequency`: Radar (carrier) frequency [Hz]
- `azimuth_steering_rate`: Azimuth steering rate for IW and EW modes [degrees/s].

# Input
- `meta_dict[dict]`: a dictionary of the metadata.

# Output
- `Sentinel1ProductInformation[structure of Sentinel1ProductInformation]`
"""
function Sentinel1ProductInformation(meta_dict)::Sentinel1ProductInformation
    pass = meta_dict["product"]["generalAnnotation"]["productInformation"]["pass"]
    timeliness_category = meta_dict["product"]["generalAnnotation"]["productInformation"]["timelinessCategory"]
    platform_heading = meta_dict["product"]["generalAnnotation"]["productInformation"]["platformHeading"]
    projection = meta_dict["product"]["generalAnnotation"]["productInformation"]["projection"]
    range_sampling_rate = meta_dict["product"]["generalAnnotation"]["productInformation"]["rangeSamplingRate"]
    radar_frequency = meta_dict["product"]["generalAnnotation"]["productInformation"]["radarFrequency"]
    azimuth_steering_rate = meta_dict["product"]["generalAnnotation"]["productInformation"]["azimuthSteeringRate"]

    platform_heading = parse(Float64,platform_heading)
    range_sampling_rate = parse(Float64, range_sampling_rate)
    radar_frequency = parse(Float64, radar_frequency)
    azimuth_steering_rate = parse(Float64, azimuth_steering_rate)

    return Sentinel1ProductInformation(pass,
        timeliness_category,
        platform_heading,
        projection,
        range_sampling_rate,
        radar_frequency,
        azimuth_steering_rate)
end

In [ ]:
meta_dict["product"]["imageAnnotation"]["processingInformation"]

In [7]:
meta_dict["product"]["imageAnnotation"]["processingInformation"]

OrderedCollections.OrderedDict{Any, Any} with 29 entries:
  "rawDataAnalysisUsed"     => "true"
  "orbitDataFileUsed"       => "true"
  "attitudeDataFileUsed"    => "false"
  "rxVariationCorrectionAp… => "true"
  "antennaElevationPattern… => "true"
  "antennaAzimuthPatternAp… => "true"
  "antennaAzimuthElementPa… => "true"
  "dcMethod"                => "Data Analysis"
  "dcInputData"             => "Range Compressed"
  "rangeSpreadingLossCompe… => "true"
  "srgrConversionApplied"   => "true"
  "detectionPerformed"      => "true"
  "thermalNoiseCorrectionP… => "false"
  "rfiMitigationPerformed"  => "BasedOnNoiseMeas"
  "rfiMitigationDomain"     => "TimeAndFrequency"
  "chirpSource"             => "Nominal"
  "pgSource"                => "Extracted"
  "rrfSpectrum"             => "Extended Tapered"
  "applicationLutId"        => "IW_Default"
  ⋮                         => ⋮

In [ ]:
pass = meta_dict["product"]["generalAnnotation"]["productInformation"]["pass"]
timeliness_category = meta_dict["product"]["generalAnnotation"]["productInformation"]["timelinessCategory"]
platform_heading = meta_dict["product"]["generalAnnotation"]["productInformation"]["platformHeading"]
projection = meta_dict["product"]["generalAnnotation"]["productInformation"]["projection"]
range_sampling_rate = meta_dict["product"]["generalAnnotation"]["productInformation"]["rangeSamplingRate"]
radar_frequency = meta_dict["product"]["generalAnnotation"]["productInformation"]["radarFrequency"]
azimuth_steering_rate = meta_dict["product"]["generalAnnotation"]["productInformation"]["azimuthSteeringRate"]


In [4]:
meta_dict["product"]["generalAnnotation"]

OrderedCollections.OrderedDict{Any, Any} with 9 entries:
  "productInformation"      => OrderedCollections.OrderedDict{Any, Any}("pass"=…
  "downlinkInformationList" => OrderedCollections.OrderedDict{Any, Any}(:count=…
  "orbitList"               => OrderedCollections.OrderedDict{Any, Any}(:count=…
  "attitudeList"            => OrderedCollections.OrderedDict{Any, Any}(:count=…
  "rawDataAnalysisList"     => OrderedCollections.OrderedDict{Any, Any}(:count=…
  "replicaInformationList"  => OrderedCollections.OrderedDict{Any, Any}(:count=…
  "noiseList"               => OrderedCollections.OrderedDict{Any, Any}(:count=…
  "terrainHeightList"       => OrderedCollections.OrderedDict{Any, Any}(:count=…
  "azimuthFmRateList"       => OrderedCollections.OrderedDict{Any, Any}(:count=…

Sentinel1ImageInformation(DateTime("2022-06-12T17:33:29"), DateTime("2022-06-12T17:33:54"), DateTime("2022-06-12T17:19:35"), 15, 0.005336815623651522, "16 bit Unsigned Integer", 10.0, 10.0, 26316.0, 16683.0, 486.4863102995529, 0.005336815623651522, 38.70540548481883, 0.001498521330831371)

In [9]:
typeof(im.productFirstLineUtcTime)

DateTime

In [15]:
DateTime(meta_dict["product"]["imageAnnotation"]["imageInformation"]["productFirstLineUtcTime"][1:19])

2022-06-12T17:33:29

In [31]:
meta_dict["product"]["imageAnnotation"]["imageInformation"]

OrderedCollections.OrderedDict{Any, Any} with 19 entries:
  "productFirstLineUtcTime" => "2022-06-12T17:33:29.384150"
  "productLastLineUtcTime"  => "2022-06-12T17:33:54.382483"
  "ascendingNodeTime"       => "2022-06-12T17:19:35.492551"
  "anchorTime"              => "2022-06-12T17:27:35.683514"
  "productComposition"      => "Slice"
  "sliceNumber"             => "15"
  "sliceList"               => OrderedCollections.OrderedDict{Any, Any}(:count=…
  "slantRangeTime"          => "5.336815623651522e-03"
  "pixelValue"              => "Detected"
  "outputPixels"            => "16 bit Unsigned Integer"
  "rangePixelSpacing"       => "1.000000e+01"
  "azimuthPixelSpacing"     => "1.000000e+01"
  "azimuthTimeInterval"     => "1.498521330831371e-03"
  "azimuthFrequency"        => "4.864863102995529e+02"
  "numberOfSamples"         => "26316"
  "numberOfLines"           => "16683"
  "zeroDopMinusAcqTime"     => "3.491769190000000e+02"
  "incidenceAngleMidSwath"  => "3.870540548481883e+01"
  

In [27]:
meta_dict["product"]["imageAnnotation"]["imageInformation"]["imageStatistics"]["outputDataStdDev"]

OrderedCollections.OrderedDict{Any, Any} with 2 entries:
  "re" => "4.550701e+01"
  "im" => "0.000000e+00"

In [30]:
meta_dict["product"]["imageAnnotation"]["imageInformation"]["sliceList"]["slice"][1]

OrderedCollections.OrderedDict{Any, Any} with 3 entries:
  "sliceNumber"      => "1"
  "sensingStartTime" => "2022-06-12T17:27:36.474464"
  "sensingStopTime"  => "2022-06-12T17:28:06.803269"

In [20]:
meta_dict["product"]["imageAnnotation"]["imageInformation"]["sliceList"]["slice"][1]

OrderedCollections.OrderedDict{Any, Any} with 3 entries:
  "sliceNumber"      => "1"
  "sensingStartTime" => "2022-06-12T17:27:36.474464"
  "sensingStopTime"  => "2022-06-12T17:28:06.803269"

In [12]:
meta_dict["product"]["imageAnnotation"]["processingInformation"]

OrderedCollections.OrderedDict{Any, Any} with 29 entries:
  "rawDataAnalysisUsed"     => "true"
  "orbitDataFileUsed"       => "true"
  "attitudeDataFileUsed"    => "false"
  "rxVariationCorrectionAp… => "true"
  "antennaElevationPattern… => "true"
  "antennaAzimuthPatternAp… => "true"
  "antennaAzimuthElementPa… => "true"
  "dcMethod"                => "Data Analysis"
  "dcInputData"             => "Range Compressed"
  "rangeSpreadingLossCompe… => "true"
  "srgrConversionApplied"   => "true"
  "detectionPerformed"      => "true"
  "thermalNoiseCorrectionP… => "false"
  "rfiMitigationPerformed"  => "BasedOnNoiseMeas"
  "rfiMitigationDomain"     => "TimeAndFrequency"
  "chirpSource"             => "Nominal"
  "pgSource"                => "Extracted"
  "rrfSpectrum"             => "Extended Tapered"
  "applicationLutId"        => "IW_Default"
  ⋮                         => ⋮

In [ ]:
["generalAnnotation"]

# Headr

In [ ]:
function get_reference_time(meta_dict)::DateTime
    start_time = meta_dict["product"]["adsHeader"]["startTime"]
    reference_time = DateTime(start_time[1:19])
    return reference_time
end

function parse_delta_time(time_string::String,reference_time::DateTime)
    milliseconds = Dates.value( DateTime(time_string[1:23]) - reference_time)
    microseconds = parse(Int, time_string[24:26])
    return milliseconds /1000.0 + microseconds*  10^-6
end

In [ ]:
"""
    Sentinel1Header

returns structure of Sentinel1Header from metadata in .xml
"""
Base.@kwdef struct Sentinel1Header
    mission_id::String
    product_type::String
    polarisation::Polarisation
    swath::Swath
    mode::String
    start_time::Float64
    stop_time::Float64
    absolute_orbit_number::Int
    image_number::String
end

In [ ]:
""""
    Sentinel1Header

Constructors for the Sentinel1Header structure.

It takes a dictionary containing the full sentinel-1 swath metadata and extracts the Sentinel1Header as a structure. Input in the header file:
- `missionId`: Mission identifier for this data set.
- `productType`: Product type for this data set.
- `polarisation`: Polarisation for this data set.
- `swath`: Swath identifier for this data set. This element identifies the swath that applies to all data contained within this data set. The swath identifier "EW" is used for products in which the 5 EW swaths have been merged. Likewise, "IW" is used for products in which the 3 IW swaths have been merged.
- `mode`: Sensor mode for this data set.
- `start_time`: Zero Doppler start time of the output image [UTC].
- `stop_time`: Zero Doppler stop time of the output image [UTC].
- `absolute_orbit_number`: Absolute orbit number at data set start time.
- `image_number`: Image number. For WV products the image number is used to distinguish between vignettes. For SM, IW and EW modes the image number is still used but refers instead to each swath and polarisation combination (known as the 'channel') of the data.

# Input
- `meta_dict[dict]`: a dictionary of the metadata.

# Output
- `Sentinel1Header[structure of Sentinel1Header]`

"""
function Sentinel1Header(meta_dict)::Sentinel1Header

    reference_time = get_reference_time(meta_dict)



    missionId = meta_dict["product"]["adsHeader"]["missionId"]
    productType = meta_dict["product"]["adsHeader"]["productType"]
    polarisation = convert_to_polarisation(meta_dict["product"]["adsHeader"]["polarisation"])
    swath = meta_dict["product"]["adsHeader"]["swath"]
    mode = meta_dict["product"]["adsHeader"]["mode"]
    start_time = meta_dict["product"]["adsHeader"]["startTime"]
    stop_time = meta_dict["product"]["adsHeader"]["stopTime"]
    absolute_orbit_number = meta_dict["product"]["adsHeader"]["absoluteOrbitNumber"]
    image_number = meta_dict["product"]["adsHeader"]["imageNumber"]
    stop_time = parse_delta_time(stop_time,reference_time)
    start_time = parse_delta_time(start_time,reference_time)
    absolute_orbit_number = parse(Int, absolute_orbit_number)
    swath = convert_to_swath(swath)


    header = Sentinel1Header(missionId,
        productType,
        polarisation,
        swath,
        mode,
        start_time,
        stop_time,
        absolute_orbit_number,
        image_number)
    return header
end


In [ ]:
header = Sentinel1Header1(meta_dict)

In [ ]:
meta_dict["product"]["adsHeader"]["polarisation"]

In [ ]:
Sentinel1ProductInformation(meta_dict)

In [ ]:
Sentinel1ImageInformation(meta_dict)

In [ ]:

Sentinel1SwathTiming(meta_dict)

In [ ]:
burst = meta_dict["product"]["adsHeader"]

In [ ]:
meta_dict["product"]["generalAnnotation"]["rawDataAnalysisList"]["rawDataAnalysis"]

In [ ]:
["geolocationGrid"]

In [ ]:
meta_dict["product"]["generalAnnotation"]

In [ ]:
Base.@kwdef struct Sentinel1Header
    mission_id::String
    product_type::String
    polarisation::Polarisation
    mission_data_take_id::Int
    swath::Int
    mode::String
    start_time::Float64
    stop_time::Float64
    absolute_orbit_number::Int
    image_number::String
end

In [ ]:
burst = meta_dict["product"]["swathMerging"]["swathMergeList"]["swathMerge"]

In [ ]:
["swathTiming"]["burstList"]["burst"][burst_number]

In [ ]:
get_sentinel1_burst_information(meta_dict,reference_time)

In [ ]:
Sentinel1GeolocationGrid(meta_dict,reference_time)

In [ ]:
#dic1 = getDictofXml(pa[1])
#dic2 = getDictofXml(pa[2])
#pa = getAnnotationPaths("/Users/kaaso/Documents/coding/JuliaEO2024/data/S1A_IW_GRDH_1SDV_20220612T173329_20220612T173354_043633_05359A_EA25.SAFE")

In [ ]:
#dic1 = getDictofXml(pa[1])
#dic2 = getDictofXml(pa[2])
#

In [ ]:
Sentinel1Header(meta_dict,reference_time)

In [ ]:
metadata = Sentinel1MetaData(pa[1])

In [ ]:

get_polarisation(meta_data::Sentinel1MetaData) = meta_data.header.polarisation
get_range_sampling_rate(meta_data::Sentinel1MetaData) = meta_data.product.range_sampling_rate
get_azimuth_frequency(meta_data::Sentinel1MetaData) = meta_data.image.azimuth_frequency
get_slant_range_time_seconds(meta_data::Sentinel1MetaData) = meta_data.image.slant_range_time_seconds
get_time_range(meta_data::Sentinel1MetaData) = (meta_data.header.start_time, meta_data.header.stop_time)
get_reference_time(meta_data::Sentinel1MetaData) = meta_data.reference_time
get_incidence_angle_mid_degrees(meta_data::Sentinel1MetaData)= meta_data.image.incidence_angle_mid_swath



## implement interface
get_metadata(image::Sentinel1SLC) = image.metadata;
get_data(image::Sentinel1SLC) = image.data;
is_deramped(image::Sentinel1SLC) = image.deramped;

In [ ]:



function get_burst_start_times(meta_data::Sentinel1MetaData)
    return [element.azimuth_time for element in meta_data.bursts]
end

function get_burst_mid_times(meta_data::Sentinel1MetaData)
    return get_burst_start_times(meta_data) .+ get_burst_duration(meta_data)/2
end

function get_burst_end_times(meta_data::Sentinel1MetaData)
    return get_burst_start_times(meta_data) .+ get_burst_duration(meta_data)
end

function get_burst_duration(meta_data::Sentinel1MetaData)
    lines_per_burst = meta_data.swath.lines_per_burst
    azimuth_frequency = meta_data.image.azimuth_frequency
    burst_duration_in_seconds = lines_per_burst /azimuth_frequency
    return burst_duration_in_seconds
end

function get_burst_row_offset(meta_data::Sentinel1MetaData)
    azimuth_frequency = get_azimuth_frequency(meta_data)
    bursts_start_times = get_burst_start_times(meta_data)
    time_delta_bursts = bursts_start_times .- meta_data.header.start_time
    burst_row_offset = time_delta_bursts.* azimuth_frequency
    return burst_row_offset
end

function get_burst_start_row(meta_data::Sentinel1MetaData)
    lines_per_burst = meta_data.swath.lines_per_burst

    return 1 .+ [n*lines_per_burst for n in 0:(length(meta_data.bursts)-1)]

end


"""
    get_image_rows(meta_data::Sentinel1MetaData, row_from_first_burst)

Converts the row number representing a unique azimuth time, row_from_first_burst,
to the number in the full image. Note that two results are returned when the row appears in
two bursts
"""
function get_image_rows(meta_data::Sentinel1MetaData, row_from_first_burst)
    burst_row_offset = get_burst_row_offset(meta_data)
    lines_per_burst = meta_data.swath.lines_per_burst
    burst_start_row = get_burst_start_row(meta_data)

    row_in_burst = row_from_first_burst .- burst_row_offset

    is_row_in_burst =  (1 .<= row_in_burst) .& (row_in_burst .<= lines_per_burst)

    image_row = row_in_burst .+ burst_start_row .- 1

    return image_row[is_row_in_burst]
end




"""
    get_window(image::Sentinel1SLC)

Returns the window of the complete Sentinel image covered the "image"
"""
function get_window(image::Sentinel1SLC)
    rows_start = image.index_start[1]
    rows_end = size(image.data)[1] + image.index_start[1] - 1

    columns_start = image.index_start[2]
    columns_end = size(image.data)[2] + image.index_start[2] - 1

    return [[rows_start, rows_end],[columns_start, columns_end]]
end

"""
    get_burst_numbers(image::Sentinel1SLC)

Returns list of burst included in the image subset
"""
function get_burst_numbers(image::Sentinel1SLC)
    lines_per_burst = image.metadata.swath.lines_per_burst
    window = get_window(image)
    index_1_range = window[1]

    burst_start = Integer(ceil( (index_1_range[1]-1) / lines_per_burst))
    burst_end = Integer(ceil( (index_1_range[2]-1) / lines_per_burst))

    return burst_start:burst_end
end


function get_burst_mid_times(image::Sentinel1SLC)
    mid_times = get_burst_mid_times(image.metadata)
    return mid_times
end


